# 第3章 ファイル操作 & 文字列操作

### ライブラリの準備
本セクションではjqueryライクにXML/HTMLを解析する[pyquery](http://pythonhosted.org/pyquery/)を使用します。ターミナル/コマンドプロンプト上で下記コマンドを実行し、上記ライブラリをインストールしてください。

In [1]:
$ cd data-science-bootcamp
$ pip install pyquery   # pyqueryライブラリ

SyntaxError: invalid syntax (<ipython-input-1-1048dc38764a>, line 1)

---

### Q11. 文字列から日時オブジェクトへの変換
"2017年12月8日 9時23分16秒"という文字列を、datetimeオブジェクトに変換せよ。

In [ ]:
import datetime

date = datetime.datetime.strptime("2017年12月8日 9時23分16秒", '%Y年%m月%d日 %H時%M分%S秒')
print(date)

### Q12. 日時計算 & 日時表示: Hello, Mac!
「2017年12月8日9時23分16秒」から「12371日と23時間23分16秒」戻った日時を求め、"X年Y月Z日"という文字列で表示せよ。

In [ ]:
import datetime

date = datetime.datetime.strptime("2017年12月8日9時23分16秒", '%Y年%m月%d日%H時%M分%S秒')
delta = datetime.timedelta(days=12371, hours=23, minutes=23, seconds=16)
result_date = date - delta

result = datetime.datetime.strftime(result_date, '%Y年%m月%d日')
print(result)

### Q13. JSONファイルの扱い
data/file_handling_assignment/kaken2017フォルダの中には、2017年度[科学研究費助成事業（学術研究助成基金助成金／科学研究費補助金）](https://www.jsps.go.jp/j-grantsinaid/)採択プロジェクトのうち、情報学分野に関連したプロジェクトの情報について記したJSONファイルがある。各ファイルには、以下の項目が記されている：
* url: プロジェクトの詳細を示したURL
* kaken_id: プロジェクトID
* project_name: プロジェクト名
* grant_category: プロジェクトの申請カテゴリ（例: 若手研究B）
* fields: 研究分野
* keywords: プロジェクトのキーワード
* budget: プロジェクトの予算規模（円）
* principal_investigator: 研究代表者に関する情報

17K17832.jsonファイルを読み込み、プロジェクトID17K1832の研究代表者、プロジェクト名、予算規模を表示せよ。


（ヒント）jsonモジュールを使う。

In [ ]:
import json

with open("../data/file_handling_assignment/kaken2017/17K17832.json") as json_file:
    json_obj = json.load(json_file)
    print("プロジェクト名:", json_obj["project_name"])
    print("研究代表者:", json_obj["principal_investigator"]["name"])
    print("予算規模:", json_obj["budget"], "円")

### Q14. ファイル一覧の取得
data/kaken2017フォルダにあるJSONファイルの情報を使って、2017年度に各研究機関が獲得した情報学分野における研究費の合計金額を求めよ。さらに、合計獲得金額の上位20機関を表示せよ。

（ヒント）osモジュールを使う。

In [ ]:
import os
import json

budgets = {}
for line in os.listdir("../data/file_handling_assignment/kaken2017"):
    filename = line.rstrip()
    
    with open("../data/file_handling_assignment/kaken2017/%s" % filename) as json_file:
        if filename.endswith(".json"):
            json_obj = json.load(json_file)
            affiliation = json_obj["principal_investigator"]["affiliation"]
            budget = int(json_obj["budget"])
            budgets.setdefault(affiliation, 0)
            budgets[affiliation] += budget

            
sorted_budgets = sorted(budgets.items(), key=lambda x: x[1], reverse=True)
for rank, (affiliation, budget) in enumerate(sorted_budgets):
    if rank >= 20:
        break
    else:
        message = "{affiliation}\t{budget}円".format(affiliation=affiliation, budget=budget)
        print(message)

### Q15. XMLファイルのパース1
data/file_handling_assignmentディレクトリにあるbig-cities-pageviews.xmlは、日本語版Wikipediaにある政令指定都市の記事に対するアクセス数（2017/11/29 - 2017/12/04）が記載されている。

当該XMLファイルには、以下に関するノードが含まれている:
* page: 各政令指定都市に関するWikipedia記事に関する情報をまとめたノード
* pageviews: アクセス数に関する情報をまとめたノード
* pvip: ある日のアクセススに関する情報を示すノード

big-cities-pageviews.xmlを読み込み、pageノードのid属性が1642589のWikipedia記事のタイトル（都市名）を表示せよ。

（ヒント）pyqueryモジュールを利用する。

In [ ]:
from pyquery import PyQuery as pq

with open("../data/file_handling_assignment/big-cities-pageviews.xml") as xml_file:
    dom = pq(xml_file.read(), parser='xml')
    for page_dom in dom("page").items():
        page_id = page_dom.attr("pageid")
        title = page_dom.attr("title")
        
        if page_id == "1642589":
            print(page_dom.attr("title"))
            break

### Q16. XMLファイルのパース2
data/file_handling_assignmentディレクトリにあるbig-cities-pageviews.xmlを読み込み、政令指定都市の記事に対するアクセス数の合計値（2017/11/29 - 2017/12/04）を各都市ごとに表示せよ。

In [ ]:
from pyquery import PyQuery as pq

pageviews = {}

with open("../data/file_handling_assignment/big-cities-pageviews.xml") as xml_file:
    dom = pq(xml_file.read(), parser='xml')
    for page_dom in dom("page").items():
        page_id = page_dom.attr("pageid")
        title = page_dom.attr("title")
        
        for pageview_dom in page_dom("pageviews pvip").items():
            pageview = int(pageview_dom.text())
            pageviews.setdefault(title, 0)
            pageviews[title] += pageview

print(pageviews)

### Q17. 正規表現1（URL抽出）
data/file_handling_assignmentディレクトリにあるhamamtsu.txtは、[浜松市に関するWikipedia記事](https://ja.wikipedia.org/wiki/%E6%B5%9C%E6%9D%BE%E5%B8%82)について[MediaWiki記法](https://ja.wikipedia.org/wiki/Help:早見表)で記述されたテキストである（2017年12月6日時点の内容）。hamamatsu.txtからWikipediaサイトからの外部リンク（httpもしくはhttpsから始まるURL）をすべて取得せよ。

In [ ]:
import re

urls = []
with open("../data/file_handling_assignment/hamamatsu.txt") as h_file:
    content = h_file.read()
    reg = re.compile(r"https?://[\w/:%#\$&\?\(\)~\.=\+\-]+")
    urls = reg.findall(content)
    
for url in urls:
    print(url)

### Q18. 正規表現2（カテゴリ抽出）
[Category:静岡県の市町村](https://ja.wikipedia.org/wiki/Category:%E9%9D%99%E5%B2%A1%E7%9C%8C%E3%81%AE%E5%B8%82%E7%94%BA%E6%9D%91)のように、Wikipediaではカテゴリが割り当てられた記事がある。[MediaWiki記法](https://ja.wikipedia.org/wiki/Help:早見表)を参考に、data/file_handling_assignmentディレクトリにあるhamamtsu.txt（浜松市に関するWikipedia記事）に割り当てられたカテゴリ名を抽出せよ（[言語処理100本ノック Q22から出題](http://www.cl.ecei.tohoku.ac.jp/nlp100/)）。

In [ ]:
import re

categories = []
with open("../data/file_handling_assignment/hamamatsu.txt") as h_file:
    content = h_file.read()
    reg = re.compile(r"\[\[Category:(.+?)\]\]")
    categories = reg.findall(content)
    categories = [re.sub(r'\|.+', '', c) for c in categories]
    
for category in categories:
    print(category)

### Q19. 正規表現3（テンプレート予測 & 抽出）
data/file_handling_assignmentディレクトリにあるhamamtsu.txt（浜松市に関するWikipedia記事）中に、浜松市における1-12月中の平均最高気温、平均最低気温について記された箇所がある。各月の平均最高気温、最低気温はあるフォーマットに基づき記述されている。

各月の平均最高気温、最低気温を正規表現を用いて抽出し、hamamatsu_temperature.tsvという名前のTSVファイルに保存せよ。なお、TSVファイルのフォーマットは以下の様式に従うこと。

|type|month|temperature|
|----|---- |
|最高|Jan|10.1|
|...|...|...|
|最低|Jan|2.5|
|...|...|...|

In [ ]:
import re

temperatures = []
with open("../data/file_handling_assignment/hamamatsu.txt") as h_file:
    content = h_file.read()
    reg = re.compile(r"\|([A-Z]?[a-z]{2}) (high|low) C = (.+?)\n")
    temperatures = reg.findall(content)

with open("../data/output/hamamatsu_temperature.tsv", "w+") as t_file:
    t_file.write("type\tmonth\ttemperature\n")
    for _month, _type, _temp in temperatures:
        if _type == "high":
            t_file.write("最高\t{_month}\t{_temp}\n".format(_month=_month, _temp=_temp))
        else:
            t_file.write("最低\t{_month}\t{_temp}\n".format(_month=_month, _temp=_temp))            

### Q20. 対訳 by 文字列置換
data/file_handling_assignmentディレクトリにあるdictionary.csvには、ある言語の語彙（other_languageカラム）と英語の語彙（englishカラム）の対応関係が1行ごとに記されている。dictonary.tsvを用いて、以下の文章を英語に変換せよ。なお、dictionary.tsvの1行目はヘッダーであり、2行目以降に単語の対応関係が記されていることに注意せよ。なお、変換後の文字列はすべて小文字で構成されていても問題ない。

> Herzlichen Glückwunsch zum Erreichen der zwanzig Aufgaben! Nächsten Aufgaben sind praktischer für Ihre Forschungsaktivitäten. Habe Spaß!

In [ ]:
import csv

target_sentence = "Herzlichen Glückwunsch zum Erreichen der zwanzig Aufgaben! Nächsten Aufgaben sind praktischer für Ihre Forschungsaktivitäten. Habe Spaß!"
target_sentence = target_sentence.lower()

with open("../data/file_handling_assignment/dictionary.csv") as dict_file:
    reader = csv.reader(dict_file)
    header = next(reader)
    
    for o_word, e_word in reader:
        target_sentence = target_sentence.replace(o_word, e_word)
        
print(target_sentence)
        